#Very Important Spark Learning - BY LEARNING this PROGRAM - WE BECOME A DATA ENGINEER (DATA CURATION DEVELOPER & DATA ANALYST)
Simply say- We are going to learn...
next level of SQL (Spark SQL) + Python Function based programming (Framework of Spark DSL) + Datawarehouse (Datalake+Lakehouse) -> Transformation & Analytics

##**1. Data Munging** - (Cleanup) Process of transforming and mapping data from Raw form into Tidy(usable) format with the intent of making it more appropriate and valuable for a variety of downstream purposes such for further Transformation/Enrichment, Egress/Outbound, analytics, Datascience/AI application & Reporting

**Passive Data Munging** - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns. <br>

**Active Data Munging**
1. Combining Data + Schema Evolution/Merging/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

###a. Passive Data Munging - 
- Visible - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.

####1. Visibily/Manually opening the file we found couple of data patterns (Manual Exploratory Data Analysis)
- It is a Structured data with comma seperator (CSV)
- No Header, No comments, footer is there in the data
- Total columns are (seperator + 1)
- Data Quality 
- - Null columns are there
- - duplicate rows
- - format issues are there (age is not in number format eg. 7-7)
- - Uniformity issues (Artist, artist)
- - Number of columns are more or less than the expected
- eg. 4000011,Francis,McNamara,47,Therapist,NewYork & 4000014,Beth,Woodard,65
- - Identification of data type

####2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description).


In [0]:
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.show(20,False)
display(rawdf1.take(20))
display(rawdf1.sample(.1))

In [0]:
#Important passive EDA structure functions we can use
rawdf1.printSchema()#I am realizing the id & age columns are having some non numerical values (supposed to be numeric)
print(rawdf1.columns)#I am understanding the column numbers/order and the column names
print(rawdf1.dtypes)#Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
for i in rawdf1.dtypes:
    if i[1]=='string':
        print(i[0])

print(rawdf1.schema)#To identify the structure of the data in the StructType and StructField format

In [0]:
#Important passive EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - 
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data",rawdf1.count())
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

###**b. Active Data Munging**
1. Combining Data + Schema Evolution/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

In [0]:
from pyspark.sql.session import SparkSession#15lakhs
spark=SparkSession.builder.appName("WD36 - ETL Pipeline - Bread & Butter").getOrCreate()#3 lakhs LOC by Databricks (for eg. display, delta, xml)

####1. Combining Data + Schema Evolution/Merging (Structuring)

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified")
#2. Multiple files (with same or different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified","/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/","/Volumes/workspace/wd36schema/ingestion_volume/staging/"],recursiveFileLookup=True,pathGlobFilter="custsm*")


In [0]:
#Active Data munging...
#When you go for Schema Merging/Melting and Schema Evolution?
#Schema Merging/Melting (unionByName,allowMissingColumns)- If we get multiple files
#Schema Evolution (orc/parquet with mergeSchema) - If no. of columns are keeps added by the source system
#when we know structure of the file already - schema merge/ schema not known earlier  - schema evolution

#COMBINING OR SCHEMA MERGING or SCHEMA MELTING of Data from different sources(Important interview question also as like schema evolution...)
#4. Multiple files with different structure in multiple paths or sub paths
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)
rawdf_merged=rawdf1.union(rawdf2)#Use union only if the dataframes are having same columns in the same order with same datatype..
display(rawdf_merged)
#Expected right approach to follow
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
display(rawdf_merged)

#Here, we are merging two files because both are in CSV format. If one file is CSV and the other file is in a different format, what should we do in this scenario? it will be handled automatically
#rawdf2.write.json("/Volumes/workspace/wd36schema/ingestion_volume/staging/csvjson")
rawdf3=spark.read.json("/Volumes/workspace/wd36schema/ingestion_volume/staging/csvjson")
rawdf_merged=rawdf_merged.unionByName(rawdf3,allowMissingColumns=True)
display(rawdf_merged)#Expected dataframe to proceed further munging on a single dataframe

Just for the simple learning of schema evolution & schema merging/melting<br>
Schema merging/melting<br>
1,rajeshwari day1(source1)<br>
1,rajeshwari,30 day1(source2)<br>

Schema evolution<br>
1,rajeshwari day1<br>
1,rajeshwari,30 day2<br>

Output is same in both cases...<br>
id,name,age<br>
1,rajeshwari,null<br>
1,null,30<br>

####2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='permissive')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)

#####Validation

In [0]:
#method3 best methodology of applying active data munging
#Validation by doing cleansing (not at the time of creating Dataframe, rather we will clean and scrub subsequently)...
struttype1=StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)

#####Cleansing
It is a process of cleaning/removing/deleting unwanted data

In [0]:
#We already know how to do cleansing applying the strict Structure on method1 and method2
#Important na functions we can use to do cleansing
cleanseddf=rawdf1.na.drop(how="any")#This function will drop any column in a given row with null otherwise this function returns rows with no null columns
display(cleanseddf.where("age is null"))#any one column contains null will be cleaned
display(rawdf1.where("age is null"))
cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["lastname","profession"])#4000004,Gretchen,,66,
display(cleanseddf)

#####Scrubbing
It is a process of polishing/fine tuning/scrubbing/meaningful conversion the data in a usable format

In [0]:
cleanseddf=rawdf1.na.fill('not provided',subset=["lastname"])#4000004,Gretchen,,66,
display(cleanseddf)

1. how to create pipelines using different data processing techniques by connecting with different sources/targets
2. how to Standardize/Modernization/Industrializing the code and how create/consume generic/reusable functions & frameworks
3. Testing (Unit, Peer Review, SIT/Integration, Regression, User Acceptance Testing), Masking engine,
4. Reusable transformation(munge_data, optimize_performance),
5. Quality suite/Data Profiling/Audit engine (Reconcilation) (Audit framework), Data/process Observability

6. how terminologies/architecture/submit jobs/monitor/log analysis/packaging and deployment ...
7. performance tuning
8. Deploying spark applications in Cloud & other Distributions like Hortonworks/Cloudera/Databricks
9. Creating cloud pipelines using spark SQL programs & Cloud native tools

What is the importance of learning this program or How this can address interview questions..?
VERY VERY IMPORTANT PROGRAM IN TERMS OF EXPLAINING/SOLVING PROBLEMS GIVEN IN INTERVIEW ,
WITH THIS ONE PROGRAM YOU CAN COVER ALMOST ALL DATAENGINEERING FEATURES
Tell me about the common transformations you performed,
tell me your daily roles in DE,
tell me some business logics you have writtened recently
How do you write an entire spark application,
levels/stages of DE pipelines or
have you created DE pipelines what are the transformations applied,
how many you have created or are you using existing framework or you created some framework?

'''
TRANSFORMATION & ANALYTICAL TECHNIQUES
Starting point - (Data Governance (security) - Tagging, categorization, classification, masking/filteration)
1. Data Munging - Process of transforming and mapping data from Raw form into Tidy(usable) format with the
intent of making it more appropriate and valuable for a variety of downstream purposes such for
further Transformation/Enrichment, Egress/Outbound, analytics, model application & Reporting
a. Passive - Data Discovery EDA (Exploratory Data Analytics)
(every layers ingestion/transformation/analytics/consumption) -
Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.
b. Active - Combining Data + Schema Evolution/Merging (Structuring)
c. Validation, Cleansing, Scrubbing - Identifying and filling gaps & Cleaning data to remove outliers and inaccuracies
Preprocessing, Preparation
Cleansing (removal of unwanted datasets eg. na.drop),
Scrubbing (convert of raw to tidy na.fill or na.replace),
d. Standardization, De Duplication and Replacement & Deletion of Data to make it in a usable format (Dataengineers/consumers)

2. Data Enrichment - Makes your data rich and detailed
a. Add, Remove, Rename, Modify/replace
b. split, merge/Concat
c. Type Casting, format & Schema Migration

3. Data Customization & Processing - Application of Tailored Business specific Rules
a. User Defined Functions
b. Building of Frameworks & Reusable Functions

4. Data Curation
a. Curation/Transformation
b. Analysis/Analytics & Summarization -> filter, transformation, Grouping, Aggregation/Summarization

5. Data Wrangling - Gathering, Enriching and Transfomation of pre processed data into usable data
a. Lookup/Reference
b. Enrichment
c. Joins
d. Sorting
e. Windowing, Statistical & Analytical processing
f. Set Operation

6. Data Publishing & Consumption - Enablement of the Cleansed, transformed and analysed data as a Data Product.
a. Discovery,
b. Outbound/Egress,
c. Reports/exports
d. Schema migration
'''